In [9]:
from API_requests import *
from scripts import *
import pyperclip

file = open("API_keys.txt", "r")
API_key = file.readlines()[0]
file.close()


def find_path(start_ids, end_ids, API_key):
    temp_region = "europe"
    temp_server = "euw1"
    
    initial_match_hist = get_matches_by_puuid(start_ids["puuid"], temp_region, API_key)
    initial_losers = get_loser_ids_from_match_ids(initial_match_hist,
                                                    start_ids,
                                                    temp_region,
                                                    API_key)


    def dfs(visited, graph, node):
        if node not in visited and node != end_ids["puuid"]:
            node_ids = get_ids_by_puuid(node, "euw1", API_key)
            
            visited.add(node)
            
            defender_stats = get_summoner_stats_by_id(node_ids["id"], temp_server, API_key)
            defender_rank = str_rank_to_int(defender_stats["league"], defender_stats["division"], defender_stats["league_points"])

            print("Visiting: " + node_ids["summoner_name"] + " | Rank: " + str(defender_rank))
            print(len(visited))
        
            for match in graph[node]:
                key = list(match.keys())[0]
                for player_ids in match[key]:
                    neighbour = player_ids["puuid"]
                    match_list = get_matches_by_puuid(neighbour, temp_region, API_key)
                    graph[neighbour] = get_loser_ids_from_match_ids(match_list, player_ids, temp_region, API_key)

                    chall_stats = get_summoner_stats_by_id(player_ids["summoner_id"], temp_server, API_key)
                    chall_rank = str_rank_to_int(chall_stats["league"], chall_stats["division"], chall_stats["league_points"])
                    print("Challenged by: " + player_ids["summoner_name"] + " | Rank: " + str(chall_rank))

                    if chall_rank >= defender_rank:
                        dfs(visited, graph, neighbour)


    def iterative_dfs(graph, node):
        visited = [node]
        stack = [node]

        while stack:
            node = stack[-1]
            if node not in visited and node != end_ids["puuid"]:
                visited.append(node)

            node_ids = get_ids_by_puuid(node, "euw1", API_key)
            defender_stats = get_summoner_stats_by_id(node_ids["id"], temp_server, API_key)
            defender_rank = str_rank_to_int(defender_stats["league"], defender_stats["division"], defender_stats["league_points"])
            print("Visiting: " + node_ids["summoner_name"] + " | Rank: " + str(defender_rank))
            print("LEN STACK: " + str(len(stack)))

            remove_from_stack = True
            for match in graph[node]:
                key = list(match.keys())[0]
                for player_ids in match[key]:
                    neighbour = player_ids["puuid"]
                    match_list = get_matches_by_puuid(neighbour, temp_region, API_key)
                    graph[neighbour] = get_loser_ids_from_match_ids(match_list, player_ids, temp_region, API_key)

                    chall_stats = get_summoner_stats_by_id(player_ids["summoner_id"], temp_server, API_key)
                    chall_rank = str_rank_to_int(chall_stats["league"], chall_stats["division"], chall_stats["league_points"])
                    print("Challenged by: " + player_ids["summoner_name"] + " | Rank: " + str(chall_rank))

                    if chall_rank >= defender_rank:
                        stack.append(neighbour)
                        remove_from_stack = False
                        break
            
            if remove_from_stack:
                stack.pop()

        return visited


    #visited = set()
    #dfs(visited, graph, start_ids["puuid"])
    graph = {start_ids["puuid"]: initial_losers}
    x = iterative_dfs(graph, start_ids["puuid"])
    return graph, x


graph, x = find_path(get_ids_by_summoner_name("Boeira", "euw1", API_key),
            get_ids_by_summoner_name("MagiFelix5", "euw1", API_key),
                                    API_key)
        
    







Error code: 503
Sleeping 121s
Visiting: Boeira | Rank: 220.075
LEN STACK: 1
Challenged by: MOHAMED XII | Rank: 300.014
Visiting: MOHAMED XII | Rank: 300.014
LEN STACK: 2
Challenged by: Krystek | Rank: 300.0
Challenged by: CRYShoota | Rank: 300.087
Visiting: CRYShoota | Rank: 300.087
LEN STACK: 3
Challenged by: WG Mantiz | Rank: 230.075
Challenged by: BlackBoarSinthis | Rank: 310.0
Visiting: BlackBoarSinthis | Rank: 310.0
LEN STACK: 4
Challenged by: Haruka J | Rank: 320.001
Visiting: Haruka J | Rank: 320.001
LEN STACK: 5
Challenged by: JuicyTooN | Rank: 320.07
Visiting: JuicyTooN | Rank: 320.07
LEN STACK: 6
Challenged by: F0RTISSAX | Rank: 310.028
Challenged by: xX1CRIS1xX | Rank: 230.1
Challenged by: Antarmen | Rank: 320.005
Challenged by: Goofiest Goober | Rank: 320.021
Challenged by: Lagday Today | Rank: 310.052
Visiting: Haruka J | Rank: 320.001
LEN STACK: 5
Challenged by: JuicyTooN | Rank: 320.07
Visiting: JuicyTooN | Rank: 320.07
LEN STACK: 6
Challenged by: F0RTISSAX | Rank: 310.0

In [6]:

pyperclip.copy(str(graph1))